## ネットワークの構築
第1章ではニューロンのモデルについて、第2章ではシナプスのモデルについて学んできました。第3章ではそれらのモデルを組み合わせたネットワークを構築してみます。また、最後の節ではSNNを学習させる意義とその方針について説明します。

\section{ニューロン間の接続}
ネットワークを構成するにはあるニューロンがどのニューロンに投射しているか、どのように活動が伝搬するかを記述する必要があります。この節ではニューロン同士の間の接続関係の記述の仕方について説明します。
\subsection{全結合(Full connection)}
$i$層目のニューロンが$i+1$層目のニューロンに全て繋がっていることを\textbf{全結合(fully connected)}と言います。ただし、全てが完全に繋がっているということではなく、結合重みが0の場合は繋がっていないことを表します。なお、この結合様式は既に第2章で出てきています。全結合はANNでは入力に重み行列を乗算し、バイアスを加算するようなアフィン変換で表されますが、SNNでは入力に重み行列を乗算するだけの線形変換を用いることが主です。\par
単に重み行列を用意するだけでも(この本の内容に限るなら)問題はありませんが、重みを学習させる場合には\texttt{class}を用意しておくと取り扱いがしやすくなります。コードは次のようになります\footnote{コードは\texttt{./TrainingSNN/Models/Connections.py}に含まれます。}。このコードを\texttt{Connections.py}として\texttt{Models}ディレクトリ内に保存しておきましょう。
\begin{minted}[frame=lines, framesep=2mm, baselinestretch=1.2, bgcolor=shadecolor,fontsize=\small]{python}
class FullConnection:
    def __init__(self, N_in, N_out, initW=None):
        if initW is not None:
            self.W = initW
        else:
            self.W = 0.1*np.random.rand(N_out, N_in)
    
    def backward(self, x):
        return np.dot(self.W.T, x) #self.W.T @ x
    
    def __call__(self, x):
        return np.dot(self.W, x) #self.W @ x
\end{minted}

\subsection{2次元の畳み込み(Convolution2D connection)}
SNNではANNの1つの結合形式である\textbf{畳み込み層}(convolutional layer)を含むことがあります。全結合が通常のANNと同様であったように畳み込み層も全く同じ操作です。そのため、今回実装はしないのですが、行列計算ライブラリとしてNumPyではなく、TensorflowやPytorch, Chainer等を使う場合には畳み込み層の関数が実装されているのでそれを使うとよいでしょう。\par
念のため、2D畳み込み層の出力テンソル($H\times W \times C$のテンソル、$H, W$はそれぞれ画像の高さと幅、$C$はチャネル数)の解釈について説明しておきます。まず、1つのチャネルは同種(同系統の受容野を持つ)の$H\times W$個のニューロンの活動です。本来は「同種」ですが、空間的な不変性により「同一」と見なし、重み共有(weight sharing, weight tying)をしてスライディングウィンドウ(sliding window)の操作をすることで、ニューロンを視野全体に複製(要は1つのニューロンをコピペ)しています。実際の視覚野では近傍のニューロンの活動を受けることによる畳み込みはしていますが、重み共有\footnote{ただし、類似の遺伝子発現による初期値共有はしているかもしれないですが。}とスライディングウィンドウはしていない、ということです。

\subsection{遅延結合(Delay connection)}
実際のニューロンにおいて、シナプス前細胞での発火が瞬間的にシナプス後細胞に伝わるということはありません。これは\textbf{軸索遅延(axonal delays)}や\textbf{シナプス遅延(synaptic delay)}があるためです。ここでは発火情報の伝搬における遅延の実装について説明します。ただし、全てのニューロンの遅延が等しいとした場合のみです\footnote{遅延時間をバラバラにすると行列での取り扱いが難しくなり、for loopを用いる他にないと思うので省略します。実装したい場合はC++やJuliaなどfor loopが速い言語を用いてください。}。\par
実装は単純で、まず、行数はニューロンの数、列数は遅延時間のステップ数と同じ長さとした行列を用意します。ステップごとに最後の行にあたるベクトルを出力し、配列をずらした後\footnote{\texttt{np.roll}を用いるよりもこちらの方が速いです。}、初めの行を新しい入力で更新します。
\begin{minted}[frame=lines, framesep=2mm, baselinestretch=1.2, bgcolor=shadecolor,fontsize=\small]{python}
class DelayConnection:
    def __init__(self, N, delay, dt=1e-4):
        nt_delay = round(delay/dt) # 遅延のステップ数
        self.state = np.zeros((N, nt_delay))
        
    def __call__(self, x):
        out = self.state[:, -1] # 出力
        self.state[:, 1:] = self.state[:, :-1] # 配列をずらす
        self.state[:, 0] = x # 入力
        return out
\end{minted}
このコードも\texttt{./Models/Connections.py}に記して保存しておきましょう。\par
次に、遅延が正しく表現されているか確認してみましょう\footnote{コードは\texttt{./TrainingSNN/example\_using\_delay\_connection.py}です。また、この部分はBrian2のtutorialを参考にしました。}。まず、\texttt{Models}ディレクトリをパッケージとして認識させるために\texttt{\_\_init\_\_.py}という名称のファイルを作成し(何も書かれてなくてよいです)、\texttt{Models}ディレクトリ内に保存します。次に\texttt{Models}ディレクトリの親ディレクトリ内にこれから書くファイルを置きます。こうすることで\texttt{Models}ディレクトリ内のファイルから作成した\texttt{class}をimportすることができます。これは以降のコードでほぼ共通です。\par
さて、コードは次のようになります。初めにモデルのimportと定数の定義、モデルのインスタンスの作成、記録用配列の定義を行っています。
\begin{minted}[frame=lines, framesep=2mm, baselinestretch=1.2, bgcolor=shadecolor,fontsize=\small]{python}
from Models.Neurons import CurrentBasedLIF
from Models.Connections import DelayConnection

dt = 1e-4; T = 5e-2; nt = round(T/dt)

#モデルの定義
neuron1 = CurrentBasedLIF(N=1, dt=dt, tc_m=1e-2, tref=0, 
                          vrest=0, vreset=0, vthr=1, vpeak=1)
neuron2 = CurrentBasedLIF(N=1, dt=dt, tc_m=1e-1, tref=0,
                          vrest=0, vreset=0, vthr=1, vpeak=1)
delay_connect = DelayConnection(N=1, delay=2e-3, dt=dt)

I = 2 # 入力電流
v_arr1 = np.zeros(nt); v_arr2 = np.zeros(nt) #記録用配列

for t in tqdm(range(nt)):
    # 更新
    s1 = neuron1(I)
    d1 = delay_connect(s1)
    s2 = neuron2(0.02/dt*d1)

    # 保存
    v_arr1[t] = neuron1.v_
    v_arr2[t] = neuron2.v_

time = np.arange(nt)*dt*1e3
plt.figure(figsize=(5, 4))
plt.plot(time, v_arr1, label="Neuron1", linestyle="dashed")
plt.plot(time, v_arr2, label="Neuron2")
plt.xlabel("Time (ms)"); plt.ylabel("v") 
plt.legend(loc="upper left")
plt.show()
\end{minted}
結果は図\ref{fig:delay}のようになります。
\begin{figure}[htbp]
    \centering
    \includegraphics[scale=0.5]{figs/delay.pdf}
    \caption{ニューロン1からニューロン2へと2 msの遅延で発火が伝わる場合。}
    \label{fig:delay}
\end{figure}

\section{ランダムネットワーク}
この節ではこれまでに実装したSNNの要素を組み合わせ、重みがランダムなネットワーク(random network)を構成してみましょう。作成するネットワークは2層から成り、1層目には10個のPoissonスパイクニューロン、2層目には1個のLIFニューロンがあるとします。1層目のニューロンから2層目のニューロンへのシナプス結合には、二重指数関数型シナプスを用います。目標は2層目のニューロンの膜電位と入力電流、1層目のニューロンのラスタープロット(raster plot)\footnote{ラスタープロットはスパイク列を表す図で、各ニューロンが発火したことを点で表します。}を表示することです。\par
それではネットワークを構築してみましょう\footnote{コードは\texttt{./TrainingSNN/LIF\_random\_network.py}です。}。まず、ニューロンとシナプスのクラスを\texttt{import}し、各種定数、入力のポアソンスパイク\texttt{x}、結合重み\texttt{W}、ニューロンとシナプスのモデルの各インスタンス(\texttt{neurons}, \texttt{synapses})、記録用の配列を定義します。注意点として、先ほどと同様に実行ファイルは\texttt{Models}ディレクトリの親ディレクトリ内に置くようにしましょう。
\begin{minted}[frame=lines, framesep=2mm, baselinestretch=1.2, bgcolor=shadecolor,fontsize=\small]{python}
from Models.Neurons import CurrentBasedLIF
from Models.Synapses import DoubleExponentialSynapse

np.random.seed(seed=0)

dt = 1e-4; T = 1; nt = round(T/dt) # シミュレーション時間
num_in = 10; num_out = 1 # 入力 / 出力ニューロンの数

# 入力のポアソンスパイク
fr_in = 30 # 入力のポアソンスパイクの発火率(Hz)
x = np.where(np.random.rand(nt, num_in) < fr_in * dt, 1, 0)
W = 0.2*np.random.randn(num_out, num_in) # ランダムな結合重み

# モデル
neurons = CurrentBasedLIF(N=num_out, dt=dt, tref=5e-3,
                          tc_m=1e-2, vrest=-65, vreset=-60,
                          vthr=-40, vpeak=30)
synapses = DoubleExponentialSynapse(N=num_out, dt=dt, td=1e-2, tr=1e-2)

# 記録用配列
current = np.zeros((num_out, nt))
voltage = np.zeros((num_out, nt))
\end{minted}
次に、\texttt{for}ループ内でネットワークの流れを書き、シミュレーションを実行してみましょう。
\begin{minted}[frame=lines, framesep=2mm, baselinestretch=1.2, bgcolor=shadecolor,fontsize=\small]{python}
# シミュレーション
neurons.initialize_states() # 状態の初期化
for t in tqdm(range(nt)):
    # 更新
    I = synapses(np.dot(W, x[t]))
    s = neurons(I)

    # 記録
    current[:, t] = I
    voltage[:, t] = neurons.v_
\end{minted}
ここでは、全結合は\texttt{np.dot(W, x[t])}で表し、\texttt{synapses}の出力はシナプス後電流とします。第二章で述べたように\texttt{synapses}の出力が何を意味するのか、すなわちシナプス前細胞の神経伝達物質の放出量なのか、シナプス後細胞のチャネルの開口頻度なのかは場合によって変わるので、注意するようにしましょう。今回の場合はシナプス後細胞に注目したモデルとなっています。\par
最後にシミュレーションの結果を描画してみましょう。描画するのは前述したように2層目のニューロンの膜電位と入力電流、1層目のニューロンのラスタープロットです。
\begin{minted}[frame=lines, framesep=2mm, baselinestretch=1.2, bgcolor=shadecolor,fontsize=\small]{python}
# 結果表示
t = np.arange(nt)*dt
plt.figure(figsize=(7, 6))
plt.subplot(3,1,1)
plt.plot(t, voltage[0], color="k")
plt.xlim(0, T)
plt.ylabel('Membrane potential (mV)') 

plt.subplot(3,1,2)
plt.plot(t, current[0], color="k")
plt.xlim(0, T)
plt.ylabel('Synaptic current (pA)') 

plt.subplot(3,1,3)
for i in range(num_in):    
    plt.plot(t, x[:, i]*(i+1), 'ko', markersize=2)
plt.xlabel('Time (s)')
plt.ylabel('Neuron index') 
plt.xlim(0, T)
plt.ylim(0.5, num_in+0.5)
plt.show()
\end{minted}
これを実行した結果は図\ref{fig:random_network}のようになります。
\begin{figure}[htbp]
    \centering
    \includegraphics[scale=0.5]{figs/LIF_random_network.pdf}
    \caption{ランダムネットワークの活動. (上)2層目のニューロンの膜電位変化. (中) 2層目のニューロンへの入力電流. (下) 1層目のポアソンスパイクニューロンのラスタープロット.}
    \label{fig:random_network}
\end{figure}

\section{SNNを訓練する}
\subsection{SNNの意義}
DeepMindのアルファ碁が2017年に柯潔九段に勝利したとき、同時に両者の消費電力(エネルギー)が話題となりました。ヒトの脳の消費エネルギーは平常時で約20W, 高負荷な思考時には約21W程度しか消費しない\footnote{ということで思考時と休止時では1W程度しか変わりません。この話は有名ですが、実は自分は元の論文を知りません。}のに対し、アルファ碁(2000CPU, 300GPU)の消費電力は25万Wであったそうです。この違いはどこにあるかといえば、アルゴリズムによる部分もありますが、ハードウェアによるところが大きいです(もちろん脳というハードウェアにおけるアルゴリズムも優れていると思いますが)。そこで、ニューロンを模した低消費電力のハードウェアとして\textbf{Neuromorphic Hardware}と呼ばれるチップが開発されています。例を挙げるとIBMのTrueNorth, IntelのLoihi, マンチェスター大学(APT group)のSpiNNaker, BrainChipのAkidaなどがあります\footnote{正直に申し上げてハードの方のサーベイはできていません。}。通常のコンピュータであればSNNの方が計算量は大きいので、むしろ消費電力は高くなりますが、これらのチップは計算をハードの特性で行うので低消費電力でSNNを実行できます。そのため、SNNを高効率に学習させるアルゴリズムが研究されています。\par
単純に考えてANNは発火率コーディング(rate coding)しか使えないのに対し、SNNは発火率コーディングに加え時間的コーディング(temporal coding)も用いることができるので、将来的にはANNを超えてもよさそうなものです。しかし、SNNの学習は難しく、ANNを超える性能というところまでは至っていません (この原因はSNNはANNのように誤差逆伝搬法という高効率なアルゴリズムが使えないということです)。そのため、SNNの訓練と一口に言っても、これさえやっておけばよいというのはありません。次節ではSNNを訓練させる方針について紹介します。
\subsection{SNNを訓練する4つの方針}
現在のところ、SNNを訓練する方針は大きく分けて4つあります(Pfeiffer \& Pfeil, 2018)\footnote{元は5つの方針が列挙されていましたが、そのうちの2つを2番にまとめました。}。
\begin{enumerate}
  \item \textbf{ANNの2値化}\\
  2値化されたANN(Binarized Neural Networks)により、SNNのように膜電位はシミュレーションしないものの、高速かつ省メモリ化にANNを実行できます。出力の2値化(0 or 1または1 or $-1$)の場合に限らず、重みや勾配も量子化(quantization)する研究もあります。なお、ここでの量子化とは出力などの値を少ないbit数で表現することを意味します。
  \item \textbf{ANNをSNNに変換}\\
  従来のANNをそのまま学習させた後、重みをそのまま、あるいは係数倍してSNNに用いるという手法があります。あるいはSNNの発火特性に一致するようにした活性化関数を用いたANNを学習させる場合もあります。
  \item \textbf{誤差逆伝搬法の近似による教師あり学習}\\
  ANNの学習に用いる誤差逆伝搬法(back-propagation)を近似し、SNNに教師あり学習を行います。
  \item \textbf{局所的な学習規則による教師なし学習}\\
  STDP(spike-timing dependent plasticity)などの学習規則で教師なし学習を行います。
\end{enumerate}
1番目の方法については触れませんが、この本では2番目の方針を第4章で、3番目の方針を第5章で、4番目の方針を第6章で扱います。さらに特殊なネットワークであるReservoir computingの学習を第7章で扱います。